# RAG系统基础:RAG系统基础：从零开始构建检索增强生成系统

RAG 系统解决的问题： **模型训练知识截至日期**和**领域专业知识的缺乏**

例如：在企业中询问 我们公司2025年Q3季度的销售数据是多少？（公司内部文件）

# RAG系统的组成

一个完整的RAG系统由三个核心组件构成：

索引阶段 Indexing -> 检索阶段 Retrieval -> 生成阶段 Generation

## Indexing

将文档转换为可搜索的向量格式

核心步骤：
    1. 文档加载 Document Loading
    2. 文本分块 Text Splitting
    3. 向量化 Vectorization
    4. 存储 Storage

## Retrieval

根据用户查询找到最相关的文档片段

核心步骤：
    1. 查询向量化
    2. 相似度计算
    3. 结果排序

## Generation

基于检索到的信息生成准确答案

核心步骤：
    1. 构建提示词
    2. LLM生成
    3. 答案返回

# 配置环境，导入核心包

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv("DASHSCOPE_API_KEY")

from langchain_qwq import ChatQwQ
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

# 索引 Indexing - 构建知识库
核心步骤：

1. 文档加载 Document Loading
2. 文本分块 Text Splitting
3. 向量化 Vectorization
4. 存储 Storage

## 步骤1：文档加载

In [ ]:
# 步骤1：首先需要加载文档。LangChain支持多种文档源
from langchain_community.document_loaders import(
    WebBaseLoader,      # 网页
    PyPDFLoader,        # PDF
    TextLoader,         # 文本文件
    DirectoryLoader,    # 目录
    CSVLoader,          # CSV
)

# Exa: Loading Webpage
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

print(f"加载了 {len(docs)} 个文档")
print(f"第一个文档长度： {(len(docs[0].page_content))} 字符")

## 步骤2：文本分块

为什么需要分块？
1. LLM上下文限制：大多数LLM有最大token限制（如GPT-4的8K/32K）
2. 提高检索精度：小块文本更容易匹配特定查询
3. 降低成本：只检索和处理相关部分

不同分块策略
```python
# 字符分块（最基础）
CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Token分块（精确控制token数量）
from langchain.text_splitter import TokenTextSplitter
TokenTextSplitter(chunk_size=256, chunk_overlap=50)

# 语义分块（基于语义相似度）
from langchain_experimental.text_splitter import SemanticChunker
SemanticChunker(OpenAIEmbeddings())

# 递归分块（智能识别段落）
RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]  # 优先按段落分割
)
```

In [ ]:
# 创建文本分块器
# 递归分块（智能识别段落）
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,            # 每块最大字符数 推荐值 500-1500，太小 -> 上下文不足，太大 -> 匹配不精准
    chunk_overlap = 200,          # 块之间的重叠 10-20% 大小，保持上下文连贯性
    length_function = len,        # 长度计算函数
    is_separator_regex = False,
    separators=["\n\n", "\n", " ", ""]  # 分隔符：优先按段落分割，段落 > 行 > 词
)

# 分块文档
splits = text_splitter.split_documents(docs)

print(f"原始文档: {len(docs)} 个")
print(f"分块后: {len(splits)} 个")
print(f"\n第一个分块示例:\n{splits[0].page_content[:30]}...")

## 步骤3：向量化
将文本转换为数学向量，这是语义搜索的核心

In [ ]:
# 创建嵌入模型
embeddings_model = DashScopeEmbeddings(
    model="text-embedding-v4",
)

# Exa: 向量化一段文字
text = "RAG是一种强大的AI技术"
new_vector_text = embeddings_model.embed_query(text)

print(f"文本: {text}")
print(f"向量维度: {len(new_vector_text)}")
print(f"向量前5个值: {new_vector_text[:5]}")

## 步骤4：存储

向量数据库选择：

| **数据库**  | **类型** | **优势**      | **适用场景**   |
|----------|--------|-------------|------------|
| Chroma   | 嵌入式    | 简单易用，无需额外服务 | 开发、小规模应用 ⭐ |
| Pinecone | 云服务    | 高性能，托管服务    | 生产环境       |
| Weaviate | 自建     | 功能丰富，开源     | 大规模部署      |
| FAISS    | 库      | 速度快，Meta开源  | 研究和原型      |

In [ ]:
# 创建向量数据库
vectorstore = Chroma.from_documents(
    documents=splits,           # 分块后的文档
    embedding=embeddings_model,       # 嵌入模型
    persist_directory="./chroma_db"  # 持久化目录
)

print(f"✅ 向量数据库创建成功！")
print(f"   存储了 {len(splits)} 个文档块")
print(f"   持久化路径: ./chroma_db_")

## 完整流程 + 封装

In [ ]:
def create_vectorstore(url: str, chunk_size: int = 1000):
    """创建向量数据库"""

    # 1. 加载文档
    loader = WebBaseLoader(url)
    docs = loader.load()

    # 2. 分块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size * 0.1)
    )
    splits = text_splitter.split_documents(docs)

    # 3. 向量化并存储
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=DashScopeEmbeddings(model="text-embedding-v4"),
        persist_directory=f"./chroma_db_{url.split('/')[-1]}"
    )

    return vectorstore

# 使用
vectorstore = create_vectorstore(
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
)